In [2]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from pathlib import Path
import pandas as pd
from io import StringIO


In [3]:
url_df = 'https://fbref.com/en/comps/9/2022-2023/schedule/2022-2023-Premier-League-Scores-and-Fixtures'

In [4]:
# Request page content and parse with BeautifulSoup
response = requests.get(url_df)
if response.status_code != 200:
    raise Exception(f"Failed to fetch data from {url_df}. HTTP Status Code: {response.status_code}")
    
soup = BeautifulSoup(response.content, 'html.parser')

# Locate the table and read it into pandas
table = soup.find('table')
if table is None:
    raise Exception("No table found on the webpage.")

# Convert the table to a string and read it using pandas
df = pd.read_html(str(table))[0]

# Flatten multi-level columns and clean them up
df.columns = [' '.join(col).strip() if isinstance(col, tuple) else col.strip() for col in df.columns]
df = df.reset_index(drop=True)

# Remove spaces in column names and drop fully empty rows
df.columns = [col.replace(" ", "_") for col in df.columns]
df = df.dropna(how='all')

# Extract hyperlinks in the "Match Report" column, if available
match_report_links = []
for row in table.find_all('tr')[1:]:  # Skip header row
    link = row.find('td', {'data-stat': 'match_report'})
    if link and link.find('a'):
        match_report_links.append('https://fbref.com' + link.find('a')['href'])

# Add hyperlinks as a new column
df['MatchReportURL'] = match_report_links

# Assign unique Match_IDs
df['Match_ID'] = range(len(df))

# Drop unnecessary columns
df = df.drop(columns=['Match_Report', 'Notes','Referee','Attendance','Venue'], errors='ignore')

# Split the score into Home and Away scores
df['Split_Score'] = df['Score'].str.split('–')  # Ensure dash is a standard hyphen
df['Home_Score'] = df['Split_Score'].str[0].astype(int)
df['Away_Score'] = df['Split_Score'].str[1].astype(int)
df = df.drop(columns=['Split_Score'], errors='ignore')

# Function to determine the winner
def get_winner(row):
    home_score = row['Home_Score']
    away_score = row['Away_Score']
    if home_score > away_score:
        return 'Home'
    elif home_score < away_score:
        return 'Away'
    else:
        return 'Draw'

# Apply the function to determine winners
df['Winner'] = df.apply(get_winner, axis=1)

# Display the DataFrame
df = df.rename(columns={'xG': 'HxG', 
                        'xG.1': 'AxG'})

df = df.reset_index(drop=True)


C:\Users\jacob.hyatt\AppData\Local\Temp\ipykernel_18036\848133236.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [5]:
df

,Wk,Day,Date,Time,Home,HxG,Score,AxG,Away,MatchReportURL,Match_ID,Home_Score,Away_Score,Winner
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,https://fbref.com/en/matches/e62f6e78/Crystal-...,0,0,2,Away
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,https://fbref.com/en/matches/6713c1dc/Fulham-L...,1,2,2,Draw
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,https://fbref.com/en/matches/09d8a999/Tottenha...,2,4,1,Home
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,https://fbref.com/en/matches/1ac96eb4/Newcastl...,3,2,0,Home
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,https://fbref.com/en/matches/82702941/Leeds-Un...,4,2,1,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,Sun,2023-05-28,16:30,Everton,1.0,1–0,0.5,Bournemouth,https://fbref.com/en/matches/94de848f/Everton-...,375,1,0,Home
376,38.0,Sun,2023-05-28,16:30,Leicester City,1.4,2–1,1.4,West Ham,https://fbref.com/en/matches/a96c9915/Leiceste...,376,2,1,Home
377,38.0,Sun,2023-05-28,16:30,Aston Villa,2.8,2–1,1.4,Brighton,https://fbref.com/en/matches/ac0e65e2/Aston-Vi...,377,2,1,Home
378,38.0,Sun,2023-05-28,16:30,Leeds United,1.5,1–4,2.2,Tottenham,https://fbref.com/en/matches/c9c73ddd/Leeds-Un...,378,1,4,Away


In [5]:
# Initialize an empty DataFrame for aggregated results
agg_df = pd.DataFrame()

# Example MatchReportURL and match_id (replace with your actual data)
url = df.loc[0, 'MatchReportURL']

# Iterate through each match in the DataFrame
for link in tqdm(range(0, 380), desc="Processing Match IDs"):

    url = df.loc[link, 'MatchReportURL']
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data for match ID {link}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table')
    num_tables = len(tables)

    headers_list = []
    footer_list = []

    # Process each table
    for i, table in enumerate(tables):
        head = table.find('thead')
        footer = table.find('tfoot')
        
        # Determine home or away team based on table index
        team_side = "H" if i < num_tables // 2 else "A"

        # Extract super-header and sub-header
        if head:
            rows = head.find_all('tr')
            if len(rows) >= 2:
                super_headers = [th.text.strip() for th in rows[0].find_all('th') if th.text.strip()]
                sub_headers = [th.text.strip() for th in rows[1].find_all('th') if th.text.strip()]
                
                # Expand super-headers based on colspan
                expanded_super_headers = []
                for th in rows[0].find_all('th'):
                    colspan = int(th.get("colspan", 1))
                    expanded_super_headers.extend([th.text.strip()] * colspan)
                
                # Combine super-header and sub-header with team side
                combined_headers = [f"{team_side}_{super_}_{sub}" for super_, sub in zip(expanded_super_headers, sub_headers)]
            else:
                combined_headers = [f"{team_side}_{th.text.strip()}" for th in rows[0].find_all('th') if th.text.strip()]
        
        # Extract footer data
        if footer:
            footer_data = [cell.text.strip() for cell in footer.find_all('td') if cell.text.strip()]
            if len(combined_headers) >= len(footer_data):
                headers_list += combined_headers[-len(footer_data):]  # Match last headers to footer data
                footer_list += footer_data

    # Create a sub DataFrame
    if headers_list and footer_list:
        sub_df = pd.DataFrame([footer_list], columns=headers_list)
        sub_df['Match_ID'] = link
        
        # Concatenate to the aggregate DataFrame
        agg_df = pd.concat([agg_df, sub_df], ignore_index=True)

    # Add a delay to avoid overwhelming the server
    time.sleep(30)


# Display aggregated DataFrame
# Define a dictionary to map old column names to new ones
column_renames = {
    'H__Tkl+Int': 'H_Defensive_Tkl+Int',
    'A__Tkl+Int': 'A_Defensive_Tkl+Int',
    'H__xA': 'H_Passing_xA',
    'A__xA': 'A_Passing_xA',
    'H__Clr': 'H_Defensive_Clr',
    'A__Clr': 'A_Defensive_Clr',
    'H__PPA': 'H_Passing_PPA',
    'A__PPA': 'A_Passing_PPA',
    'H__KP': 'H_Passing_KP',
    'A__KP': 'A_Passing_KP',
    'H__1/3': 'H_Passing_1/3',
    'A__1/3': 'A_Passing_1/3',
    'H__Min': 'H_Min'
    # Add more column renaming rules as needed
}

# Apply the renaming
agg_df = agg_df.rename(columns=column_renames)
agg_df = agg_df.loc[:, ~agg_df.T.duplicated()]

Processing Match IDs:  83%|████████▎ | 314/380 [2:43:20<24:21, 22.14s/it]  

Failed to retrieve data for match ID 313


Processing Match IDs: 100%|██████████| 380/380 [3:17:39<00:00, 31.21s/it]


In [6]:
agg_df = pd.read_csv('final_2023.csv')

In [7]:
# Initialize an empty DataFrame for aggregated results
missing_df = pd.DataFrame()

# Example MatchReportURL and match_id (replace with your actual data)
url = df.loc[0, 'MatchReportURL']

# Iterate through each match in the DataFrame
for link in tqdm([313], desc="Processing Match IDs"):

    url = df.loc[link, 'MatchReportURL']
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data for match ID {link}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table')
    num_tables = len(tables)

    headers_list = []
    footer_list = []

    # Process each table
    for i, table in enumerate(tables):
        head = table.find('thead')
        footer = table.find('tfoot')
        
        # Determine home or away team based on table index
        team_side = "H" if i < num_tables // 2 else "A"

        # Extract super-header and sub-header
        if head:
            rows = head.find_all('tr')
            if len(rows) >= 2:
                super_headers = [th.text.strip() for th in rows[0].find_all('th') if th.text.strip()]
                sub_headers = [th.text.strip() for th in rows[1].find_all('th') if th.text.strip()]
                
                # Expand super-headers based on colspan
                expanded_super_headers = []
                for th in rows[0].find_all('th'):
                    colspan = int(th.get("colspan", 1))
                    expanded_super_headers.extend([th.text.strip()] * colspan)
                
                # Combine super-header and sub-header with team side
                combined_headers = [f"{team_side}_{super_}_{sub}" for super_, sub in zip(expanded_super_headers, sub_headers)]
            else:
                combined_headers = [f"{team_side}_{th.text.strip()}" for th in rows[0].find_all('th') if th.text.strip()]
        
        # Extract footer data
        if footer:
            footer_data = [cell.text.strip() for cell in footer.find_all('td') if cell.text.strip()]
            if len(combined_headers) >= len(footer_data):
                headers_list += combined_headers[-len(footer_data):]  # Match last headers to footer data
                footer_list += footer_data

    # Create a sub DataFrame
    if headers_list and footer_list:
        sub_df = pd.DataFrame([footer_list], columns=headers_list)
        sub_df['Match_ID'] = link
        
        # Concatenate to the aggregate DataFrame
        missing_df = pd.concat([missing_df, sub_df], ignore_index=True)

    # Add a delay to avoid overwhelming the server
    time.sleep(30)


# Display aggregated DataFrame
# Define a dictionary to map old column names to new ones
column_renames = {
    'H__Tkl+Int': 'H_Defensive_Tkl+Int',
    'A__Tkl+Int': 'A_Defensive_Tkl+Int',
    'H__xA': 'H_Passing_xA',
    'A__xA': 'A_Passing_xA',
    'H__Clr': 'H_Defensive_Clr',
    'A__Clr': 'A_Defensive_Clr',
    'H__PPA': 'H_Passing_PPA',
    'A__PPA': 'A_Passing_PPA',
    'H__KP': 'H_Passing_KP',
    'A__KP': 'A_Passing_KP',
    'H__1/3': 'H_Passing_1/3',
    'A__1/3': 'A_Passing_1/3',
    'H__Min': 'H_Min'
    # Add more column renaming rules as needed
}

# Apply the renaming
missing_df = missing_df.rename(columns=column_renames)

Processing Match IDs: 100%|██████████| 1/1 [00:31<00:00, 31.86s/it]


In [8]:
merged_df = df.merge(missing_df)

In [12]:
merged_df

,Wk,Day,Date,Time,Home,HxG,Score,AxG,Away,MatchReportURL,...,A_Performance_Crs,A_Performance_Int,A_Performance_TklW,A_Performance_PKwon,A_Performance_PKcon,A_Performance_OG,A_Performance_Recov,A_Aerial Duels_Won,A_Aerial Duels_Lost,A_Aerial Duels_Won%
0,32.0,Sun,2023-04-23,14:00,Newcastle Utd,3.7,6–1,1.0,Tottenham,https://fbref.com/en/matches/dff22d13/Newcastl...,...,8,7,11,0,0,0,57,4,10,28.6


In [16]:
agg_df

,Wk,Day,Date,Time,Home,HxG,Score,AxG,Away,MatchReportURL,...,A_Carries_Dis,A_Receiving_Rec,A_Receiving_PrgR,A_Performance_2CrdY,A_Performance_Fls,A_Performance_Fld,A_Performance_PKwon,A_Performance_OG,A_Performance_Recov,A_Aerial Duels_Won%
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,https://fbref.com/en/matches/e62f6e78/Crystal-...,...,15,376,39,0,11,16,0,0,55,58.3
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,https://fbref.com/en/matches/6713c1dc/Fulham-L...,...,13,510,52,0,9,7,0,0,63,36.1
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,https://fbref.com/en/matches/09d8a999/Tottenha...,...,17,349,30,0,6,11,0,1,56,45.8
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,https://fbref.com/en/matches/1ac96eb4/Newcastl...,...,8,243,16,0,14,9,0,0,52,57.1
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,https://fbref.com/en/matches/82702941/Leeds-Un...,...,7,501,44,0,9,13,0,1,65,43.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,38.0,Sun,2023-05-28,16:30,Everton,1.0,1–0,0.5,Bournemouth,https://fbref.com/en/matches/94de848f/Everton-...,...,5,474,40,0,12,11,0,0,53,46.2
375,38.0,Sun,2023-05-28,16:30,Leicester City,1.4,2–1,1.4,West Ham,https://fbref.com/en/matches/a96c9915/Leiceste...,...,7,516,54,0,10,8,0,0,53,31.3
376,38.0,Sun,2023-05-28,16:30,Aston Villa,2.8,2–1,1.4,Brighton,https://fbref.com/en/matches/ac0e65e2/Aston-Vi...,...,9,365,33,0,16,15,0,0,46,47.8
377,38.0,Sun,2023-05-28,16:30,Leeds United,1.5,1–4,2.2,Tottenham,https://fbref.com/en/matches/c9c73ddd/Leeds-Un...,...,9,359,28,0,5,7,0,0,67,45.5


In [19]:
df2 = merged_df.loc[:, ~merged_df.columns.duplicated()]
df2


,Wk,Day,Date,Time,Home,HxG,Score,AxG,Away,MatchReportURL,...,A_Performance_Off,A_Performance_Crs,A_Performance_TklW,A_Performance_PKwon,A_Performance_PKcon,A_Performance_OG,A_Performance_Recov,A_Aerial Duels_Won,A_Aerial Duels_Lost,A_Aerial Duels_Won%
0,32.0,Sun,2023-04-23,14:00,Newcastle Utd,3.7,6–1,1.0,Tottenham,https://fbref.com/en/matches/dff22d13/Newcastl...,...,0,8,11,0,0,0,57,4,10,28.6


In [23]:
agg_df

,Wk,Day,Date,Time,Home,HxG,Score,AxG,Away,MatchReportURL,...,A_Carries_Dis,A_Receiving_Rec,A_Receiving_PrgR,A_Performance_2CrdY,A_Performance_Fls,A_Performance_Fld,A_Performance_PKwon,A_Performance_OG,A_Performance_Recov,A_Aerial Duels_Won%
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,https://fbref.com/en/matches/e62f6e78/Crystal-...,...,15,376,39,0,11,16,0,0,55,58.3
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,https://fbref.com/en/matches/6713c1dc/Fulham-L...,...,13,510,52,0,9,7,0,0,63,36.1
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,https://fbref.com/en/matches/09d8a999/Tottenha...,...,17,349,30,0,6,11,0,1,56,45.8
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,https://fbref.com/en/matches/1ac96eb4/Newcastl...,...,8,243,16,0,14,9,0,0,52,57.1
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,https://fbref.com/en/matches/82702941/Leeds-Un...,...,7,501,44,0,9,13,0,1,65,43.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,38.0,Sun,2023-05-28,16:30,Everton,1.0,1–0,0.5,Bournemouth,https://fbref.com/en/matches/94de848f/Everton-...,...,5,474,40,0,12,11,0,0,53,46.2
375,38.0,Sun,2023-05-28,16:30,Leicester City,1.4,2–1,1.4,West Ham,https://fbref.com/en/matches/a96c9915/Leiceste...,...,7,516,54,0,10,8,0,0,53,31.3
376,38.0,Sun,2023-05-28,16:30,Aston Villa,2.8,2–1,1.4,Brighton,https://fbref.com/en/matches/ac0e65e2/Aston-Vi...,...,9,365,33,0,16,15,0,0,46,47.8
377,38.0,Sun,2023-05-28,16:30,Leeds United,1.5,1–4,2.2,Tottenham,https://fbref.com/en/matches/c9c73ddd/Leeds-Un...,...,9,359,28,0,5,7,0,0,67,45.5


In [25]:
common_columns = agg_df.columns.intersection(df2.columns)

# Filter DataFrames to include only the common columns
agg_df_filtered = agg_df[common_columns]
df2_filtered = df2[common_columns]

In [27]:
agg_df2 = pd.concat([agg_df, df2_filtered]).reset_index(drop=True)

In [6]:
merged_df = df.merge(agg_df)

In [28]:
agg_df2['Match_ID']

0        0
1        1
2        2
3        3
4        4
      ... 
375    376
376    377
377    378
378    379
379    313
Name: Match_ID, Length: 380, dtype: int64

In [29]:
df_sorted = agg_df2.sort_values(by='Match_ID')

# If you want to reset the index after sorting
df_sorted = df_sorted.reset_index(drop=True)

In [30]:
df_sorted

,Wk,Day,Date,Time,Home,HxG,Score,AxG,Away,MatchReportURL,...,A_Carries_Dis,A_Receiving_Rec,A_Receiving_PrgR,A_Performance_2CrdY,A_Performance_Fls,A_Performance_Fld,A_Performance_PKwon,A_Performance_OG,A_Performance_Recov,A_Aerial Duels_Won%
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,https://fbref.com/en/matches/e62f6e78/Crystal-...,...,15,376,39,0,11,16,0,0,55,58.3
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,https://fbref.com/en/matches/6713c1dc/Fulham-L...,...,13,510,52,0,9,7,0,0,63,36.1
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,https://fbref.com/en/matches/09d8a999/Tottenha...,...,17,349,30,0,6,11,0,1,56,45.8
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,https://fbref.com/en/matches/1ac96eb4/Newcastl...,...,8,243,16,0,14,9,0,0,52,57.1
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,https://fbref.com/en/matches/82702941/Leeds-Un...,...,7,501,44,0,9,13,0,1,65,43.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,Sun,2023-05-28,16:30,Everton,1.0,1–0,0.5,Bournemouth,https://fbref.com/en/matches/94de848f/Everton-...,...,5,474,40,0,12,11,0,0,53,46.2
376,38.0,Sun,2023-05-28,16:30,Leicester City,1.4,2–1,1.4,West Ham,https://fbref.com/en/matches/a96c9915/Leiceste...,...,7,516,54,0,10,8,0,0,53,31.3
377,38.0,Sun,2023-05-28,16:30,Aston Villa,2.8,2–1,1.4,Brighton,https://fbref.com/en/matches/ac0e65e2/Aston-Vi...,...,9,365,33,0,16,15,0,0,46,47.8
378,38.0,Sun,2023-05-28,16:30,Leeds United,1.5,1–4,2.2,Tottenham,https://fbref.com/en/matches/c9c73ddd/Leeds-Un...,...,9,359,28,0,5,7,0,0,67,45.5


In [31]:
filepath = Path('final_2023.csv')  
df_sorted.to_csv(filepath, index=False)